**Importing the libraries**

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
tf.__version__

'2.9.1'

**Data Preprocessing**

--split folders

In [2]:
import splitfolders
splitfolders.ratio("alphabet letters", output="Alphabet  dataset", ratio = (.7,.2,.1), group_prefix=None, move = True)

--Preprocessing the Training set
- Data Augmantation 
- import data

In [2]:
train_datagen = ImageDataGenerator(
                                    brightness_range = (0.2, 0.8), 
                                    rescale = 1./255,
                                    zoom_range = [0.5, 1.5],
                                    channel_shift_range=150.0,
                                    horizontal_flip = True,
                                   )
training_set = train_datagen.flow_from_directory('alphabet demo dataset\\train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 3150 images belonging to 3 classes.


Preprocessing the Test set
- import validaiton data

In [3]:

val_datagen = ImageDataGenerator(rescale = 1./255)
val_set = val_datagen.flow_from_directory('alphabet demo dataset\\val',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 900 images belonging to 3 classes.


**Building the CNN**

--Initialising the CNN

In [4]:
cnn = tf.keras.models.Sequential()

Step 1 - Convolution

In [5]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

Step 2 - Pooling

In [6]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

Adding a second convolutional layer

In [7]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

Step 3 - Flattening

In [8]:
cnn.add(tf.keras.layers.Flatten())

Step 4 - Full Connection

In [9]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

Step 5 - Output Layer

In [10]:
cnn.add(tf.keras.layers.Dense(units=3, activation='softmax'))

**Training the CNN**

--Compiling the CNN

In [11]:
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

Training the CNN on the Training set and evaluating it on the Test set

In [12]:
cnn.fit(x = training_set, validation_data = val_set, epochs = 20)

Epoch 1/20
99/99 [==============================] - 20s 193ms/step - loss: 1.0737 - accuracy: 0.3854 - val_loss: 1.0720 - val_accuracy: 0.4600
Epoch 2/20
99/99 [==============================] - 19s 191ms/step - loss: 0.9747 - accuracy: 0.5079 - val_loss: 0.7252 - val_accuracy: 0.7033
Epoch 3/20
99/99 [==============================] - 21s 214ms/step - loss: 0.8872 - accuracy: 0.5794 - val_loss: 0.6548 - val_accuracy: 0.7511
Epoch 4/20
99/99 [==============================] - 26s 261ms/step - loss: 0.8335 - accuracy: 0.6162 - val_loss: 0.6719 - val_accuracy: 0.7467
Epoch 5/20
99/99 [==============================] - 20s 197ms/step - loss: 0.7791 - accuracy: 0.6419 - val_loss: 0.5007 - val_accuracy: 0.8200
Epoch 6/20
99/99 [==============================] - 16s 166ms/step - loss: 0.7353 - accuracy: 0.6638 - val_loss: 0.4047 - val_accuracy: 0.8567
Epoch 7/20
99/99 [==============================] - 14s 143ms/step - loss: 0.7168 - accuracy: 0.6848 - val_loss: 0.3786 - val_accuracy: 0.8800

Part 4 - Making a single prediction

In [13]:
training_set.class_indices

{'a': 0, 'b': 1, 'c': 2}

In [14]:
import numpy as np
from keras.preprocessing import image
test_image = tf.keras.utils.load_img('Alphabet  dataset\\test\\b\\b_4.jpg', target_size = (64, 64))
test_image = tf.keras.utils.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
print(result)

1/1 [==============================] - 0s 105ms/step
[[0. 1. 0.]]


**Create model file**

In [16]:
cnn.save('demo_cnn_model.h5')

--load and sum the model

In [17]:
model = tf.keras.models.load_model('demo_cnn_model.h5')

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)               8

**One image tests**


predict

In [19]:

import numpy as np
test_image = tf.keras.utils.load_img('Alphabet  dataset\\test\\b\\b_4.jpg', target_size = (64, 64))
test_image = tf.keras.utils.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
print(test_image)
result = model.predict(test_image)
print(result)

[[[[121. 121. 121.]
   [114. 114. 114.]
   [154. 154. 154.]
   ...
   [247. 247. 247.]
   [246. 246. 246.]
   [244. 244. 244.]]

  [[156. 156. 156.]
   [114. 114. 114.]
   [ 78.  78.  78.]
   ...
   [252. 252. 252.]
   [251. 251. 251.]
   [250. 250. 250.]]

  [[151. 151. 151.]
   [165. 165. 165.]
   [163. 163. 163.]
   ...
   [253. 253. 253.]
   [252. 252. 252.]
   [251. 251. 251.]]

  ...

  [[  2.   2.   2.]
   [  2.   2.   2.]
   [  2.   2.   2.]
   ...
   [ 37.  37.  37.]
   [ 32.  32.  32.]
   [ 29.  29.  29.]]

  [[  2.   2.   2.]
   [  2.   2.   2.]
   [  2.   2.   2.]
   ...
   [ 38.  38.  38.]
   [ 36.  36.  36.]
   [ 34.  34.  34.]]

  [[  2.   2.   2.]
   [  2.   2.   2.]
   [  2.   2.   2.]
   ...
   [ 27.  27.  27.]
   [ 28.  28.  28.]
   [ 29.  29.  29.]]]]
1/1 [==============================] - 0s 107ms/step
[[0. 1. 0.]]


final result

In [20]:
letter_position = 0
for i in result[0]:   
    if i != 0:
        print(i)
        position = list(training_set.class_indices.values()).index(letter_position)
        print(position)
        print(list(training_set.class_indices.keys())[position])
    letter_position += 1

1.0
1
b
